In [2]:
from bioprospecting import structural_search
from bioprospecting import alignment
from bioprospecting import PDB 
import os
import shutil
import glob
import pandas as pd
from Bio import PDB as pdb


In [6]:
folder = os.getcwd()

cluster_srch = folder.split("/")[-1]
path = f"/home/idg29/BSC/Immob/Proyectos/CBMs/CAZY/all_cbm_clusters/{cluster_srch}_*"

matching_folders = glob.glob(path)

if matching_folders:
    target = matching_folders[0]
    sorted_files = sorted(os.listdir(target))
    my_file = sorted_files[0]

    print(f"Accediendo a directorio {target}.")
    pdb_path = os.path.join(target, my_file)
    
    shutil.copy(pdb_path, folder)
    print(f"El archivo {my_file} ha sido movido al directorio de trabajo.")
    new_path = os.path.join(folder, my_file)   

Accediendo a directorio /home/idg29/BSC/Immob/Proyectos/CBMs/CAZY/all_cbm_clusters/0042_P01683.
El archivo P01618.pdb ha sido movido al directorio de trabajo.


In [13]:
FS_search = structural_search.foldseek.runEasySearch(target_pdb=new_path, output_folder="FoldSeek_Search", db="/home/bubbles/foldseek_databases/afsp", max_seqs=1000000, overwrite=True)

Output folder has been generated. Execute command.sh to run foldseek.


In [21]:
path = "/home/idg29/BSC/Immob/Proyectos/Try/0042/FoldSeek_Search/fs_output2.txt"

pdb_id = pd.read_csv(path, sep = "\t")

pdb_id.columns=["OG_Structure", "PDB_id", "C1", "C2", "C3", "C4", "C5", "OG_Seq", "Sim_Seq"]

In [22]:
#Dicionario con PDB_ID, Cadena de interés y secuencia.
pdb_dic = {}
pdb_list = []
for ident in pdb_id["PDB_id"]:
    chain = (ident[-1])
    pdb_dic[ident]={}
    for seq in pdb_id.loc[pdb_id["PDB_id"]==ident]["Sim_Seq"]:
        pdb_dic[ident][chain]=seq

#Lista tan solo de los PDB_ID
for PDB_ident in pdb_dic:
    pdb_list.append((PDB_ident[:4]).upper())
    

In [23]:
len(pdb_dic)

11847

In [ ]:
#Descargar PDBs
Dwnld_PDBS = PDB.retrievePDBs(pdb_list)

In [11]:
#Obtener tan solo las cadenas deseadas
PDB_fold = "/home/ivan/BSC/Immob/Proyectos/Try/0042/PDB"

if not os.path.exists(PDB_fold+"/PDB_Chain_Select"):
    os.mkdir(PDB_fold+"/PDB_Chain_Select")

parser = pdb.PDBParser(QUIET=True)
for PDBB in os.listdir(PDB_fold):
    if PDBB.endswith(".pdb"):
        for key in pdb_dic:
            if (PDBB.split(".")[0]).lower() in key:                
                structure = parser.get_structure(PDBB, os.path.join(PDB_fold, PDBB))
                chain_id = (key.split(".")[0]).split("_")[-1]
                for model in structure:
                    for chain in model:
                        if chain.id == chain_id:
                            new_structure = PDB.chainsAsStructure(chain)
                            PDB.saveStructureToPDB(new_structure, output=f"{PDB_fold}/PDB_Chain_Select/{PDBB.split('.')[0]}_{chain_id}.pdb")


In [12]:
alignment.foldseek.structuralClustering(job_folder="Clustering_Folder", models_folder="/home/ivan/BSC/Immob/Proyectos/Try/0042/PDB/PDB_Chain_Select/")


easy-cluster ..//home/ivan/BSC/Immob/Proyectos/Try/0042/PDB/PDB_Chain_Select/ output tmp -e 10.0 -c 0.9 

MMseqs Version:                     	10.941cd33
Substitution matrix                 	aa:3di.out,nucl:3di.out
Seed substitution matrix            	aa:3di.out,nucl:3di.out
Sensitivity                         	4
k-mer length                        	0
Target search mode                  	0
k-score                             	seq:2147483647,prof:2147483647
Max sequence length                 	65535
Max results per query               	300
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.9
Coverage mode                       	0
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	0
Mask residues probability           	0.999995
Mask lower cas

Input ..//home/ivan/BSC/Immob/Proyectos/Try/0042/PDB/PDB_Chain_Select/ does not exist


FileNotFoundError: [Errno 2] No such file or directory: 'Clustering_Folder/output_cluster.tsv'